In [140]:
import pandas as pd

### 2 Procesamiento de Datos
<span>Normalizar toda la información de Museos, Salas de Cine y Bibliotecas Populares, para crear una única tabla que contenga:</span>

<style>
    ul{columns:4;}
    div{margin-top:25px;}
</style>
<div class='lista'>
    <ul>
        <li>cod_localidad</li>
        <li>id_provincia</li>
        <li>id_departamento</li>
        <li>categoría</li>
        <li>provincia</li>
        <li>localidad</li>
        <li>nombre</li>
        <li>domicilio</li>
        <li>código postal</li>
        <li>número de teléfono</li>
        <li>mail</li>
        <li>web</li>
    </ul>
</div>


#### 2.1 Normalizando y consolidando columnas

**❕❕❕ Crear constants con strings de fechas y paths a los archivos correspondientes ❕❕❕**

In [124]:
# Leer csvs

# ❕❕❕ Ajustar el path a cada archivo según fecha del evento ❕❕❕
raw_df_bibliotecas = pd.read_csv('./bibliotecas/2022-October/bibliotecas-01-10-2022.csv')
raw_df_museos = pd.read_csv('./museos/2022-October/museos-01-10-2022.csv')
raw_df_cines = pd.read_csv('./cines/2022-October/cines-01-10-2022.csv')

In [125]:
def set_columns_lower(df):
    '''
    Convierte los nombres de las variables
    del dataframe a minúsculas
    '''
    df.columns = df.columns.str.lower()


In [126]:
set_columns_lower(raw_df_bibliotecas)
set_columns_lower(raw_df_museos)
set_columns_lower(raw_df_cines)

In [127]:
# Crear lista de nomenclaturas para nueva tabla
columnas = ['cod_localidad', 'id_provincia', 'id_departamento', 'categoría', 
            'provincia', 'localidad', 'nombre', 'domicilio', 'código postal', 
            'número de teléfono', 'mail', 'web']

In [128]:
# Seleccionar columnas necesarias en cada dataframe
df_bibliotecas = raw_df_bibliotecas[['cod_loc', 'idprovincia', 'iddepartamento', 'categoría', 'provincia', 'localidad', 'nombre', 'domicilio', 'cp', 'teléfono', 'mail', 'web']]
df_museos = raw_df_museos[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'telefono', 'mail', 'web']]

# Museos necesita columnas para telefono y mail
raw_df_cines[['número de teléfono', 'mail']] = 's/d'
#raw_df_cines['mail'] = 's/d'
df_cines = raw_df_cines[['cod_localidad', 'id_provincia', 'id_departamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'número de teléfono','mail' , 'web']]

In [129]:
# Renombrando columnas
def rename_cols(df, new_names):
    '''
    Crea un diccionario a partir de un zip entre las columnas actuales 
    del dataframe y una lista con sus correspondientes nuevos nombres
    Pasar como primer argumento el dataframe con las columnas a renombrar
    Pasar como segundo argumento la lista con los nuevos nombres de las columnas
    Los indices de ambos argumentos deben coincidir para que se descompaginen los nombres
    '''
    diccionario = dict(zip(df.columns, new_names))
    df = df.rename(columns=diccionario)
    return df


In [130]:
# Crear dataframes con columnas renombradas
df_bibliotecas = rename_cols(df_bibliotecas, columnas)
df_museos = rename_cols(df_museos, columnas)
df_cines = rename_cols(df_cines, columnas)

#### 2.2 Normalizando datos

In [131]:
df_bibliotecas.fillna('s/d', inplace=True)
df_museos.fillna('s/d', inplace=True)
df_cines.fillna('s/d', inplace=True)

#### 2.3 Crear tabla única

In [132]:
consolidado = pd.concat([df_bibliotecas,df_museos,df_cines], ignore_index=True)

### 3 Procesando los datos conjuntos


#### 3.1 Generar una tabla con la siguiente información:
- Cantidad de registros totales por categoría
- Cantidad de registros totales por fuente
- Cantidad de registros por provincia y categoría

In [133]:
registros_categoria = pd.DataFrame(consolidado.groupby(by= 'categoría')['categoría'].count())
registros_categoria.columns=['total_registros']

In [134]:
registros_fuente = pd.concat([raw_df_bibliotecas,raw_df_museos,raw_df_cines], ignore_index=True)
registros_fuente = pd.DataFrame(registros_fuente.groupby(by='fuente')['fuente'].count())
registros_fuente.columns=['total_registros']

In [135]:
registros_provincia_categoria = pd.DataFrame(consolidado.groupby(by=['provincia', 'categoría'])['provincia'].count())
registros_provincia_categoria.columns = ['total_registros']

#### 3.2 Procesar la información de cines
Crear una tabla que contenga:
- Provincia
- Cantidad de pantallas
- Cantidad de butacas
- Cantidad de espacios INCAA

In [137]:
tabla_cines = raw_df_cines[['provincia', 'pantallas', 'butacas', 'espacio_incaa']].copy()
tabla_cines['espacio_incaa'] = (tabla_cines['espacio_incaa'] == 'Si').astype(int)
tabla_cines = tabla_cines.groupby(by='provincia').sum()